<a href="https://colab.research.google.com/github/dbamman/anlp21/blob/main/16.ie/DependencyPatterns_TODO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook explores relation extraction by measuring common dependency paths between two entities that hold a given relation to each other -- here, the relation "born_in" between a PER entity and an GPE entity, using data from Wikipedia biographies.  Feel free to run this notebook either on your on computer or on Google Colab. If using Google Colab, install these libraries first:

In [ ]:
!pip install spacy==2.1.0

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
!pip install neuralcoref --no-binary neuralcoref

Otherwise, use your local `anlp21-spacy2` environment we set up in class on Tuesday.

In [ ]:
import re
import spacy
import neuralcoref
from collections import Counter

In [ ]:
nlp = spacy.load('en_core_web_sm')
coref = neuralcoref.NeuralCoref(nlp.vocab)
nlp.add_pipe(coref, name='neuralcoref')

In [ ]:
def get_path(one, two):
    
    """ Get dependency path between two tokens in a sentence; return None if not reachable """
    
    one_heads=[x for x in one.ancestors]
    two_heads=[x for x in two.ancestors]
    
    up_path=[]
    down_path=[]
    up_path.append(one)
    down_path.append(two)
    
    lca=None
    for head in one_heads:
        if head in two_heads:
            lca=head
            break
            
        up_path.append(head)

    for head in two_heads:
        if head == lca:
            break
    
        down_path.append(head)
   
    if lca is None:
        return None
    
    path="%s->%s<-%s" % ('->'.join(["%s" % x.dep_ for x in up_path]), lca.text, '<-'.join(["%s" % x.dep_ for x in reversed(down_path)]))
    return path

In [ ]:
def get_closest_coref(entity1, clusters, target_entity):
    
    """ Given entities e1 and mention m2 of another entity, returns the mention for e1 closest to m2 """
    
    targetCluster=None
    for chain in clusters:
        for mention in chain.mentions:
            if mention.start <= entity1.start and mention.end >= entity1.end:
                targetCluster=chain
                break

    if targetCluster is None:
        return None

    closestMention=None
    dist=100
    for mention in targetCluster:
            sentDist=abs(target_entity.sent.start-mention.sent.start)
            if sentDist < dist:
                dist=sentDist
                closestMention=mention
            if sentDist == dist and closestMention is not None:
                if abs(target_entity.start-mention.start) < abs(target_entity.start-closestMention.start):
                    closetMention=mention
    return closestMention
                

Q0. In class activity: here's [a Google spreadsheet](https://docs.google.com/spreadsheets/d/1PNDInP5JIqad9mOXwRUxGDZntvoUerX22QQcgFCJDxY/edit?usp=sharing) with the first 5 sentences from ~500 Wikipedia biographies.  If you're present in class, pick 10 rows of this spreadsheet that other students haven't already claimed and put your student ID in the "Student ID" column; then go through those 10 rows and read the document. If you can infer that a person from the "Candidate people" column was born in a place in the "Candidate places" column, list that person in the "PER BORN" column and the place in the "PLACE BORN" column.

In [ ]:
def read_training(filename):
    
    """ Read in training data for <person, place> tuples that express the "born_in" relation.
    
    -- Use coreference resolution to identity the person mention closest to the place mention.
    -- Use dependency parsing to extract the syntactic path from that person mention to the place.
    
    """
    
    data=[]
    with open(filename) as file:
        for line in file:
            cols=line.split("\t")
            idd=cols[0]
            doc=cols[1]
            pers=cols[4]
            place=cols[5].rstrip()
            
            if pers != "" and place != "":
                doc=nlp(doc)

                target_person=None
                target_place=None
                
                # Annotations are at the type level, so let's anchor them to specific mentions
                for entity in doc.ents:
                    if entity.text == pers:
                        target_person=entity
                    elif entity.text == place:
                        target_place=entity
                
                if target_person is not None and target_place is not None:
                    
                    # Use coreference to get person mention that's closest to the place (ideally in the same sentence).
                    closest_person_mention=get_closest_coref(target_person, doc._.coref_clusters, target_place)
                    if closest_person_mention is None:
                        closest_person_mention=target_person
                    
                    path=get_path(closest_person_mention.root, target_place.root)
                    
                    # if a path can be found between the two
                    if path is not None:
                        data.append((pers, place, path, target_place.sent ))
    return data     

At the end of class, save this Google sheet as a tsv in `born.tsv` and execute the `read_training` function on it to read in the <person, place> tuples.  If you're executing this on Google Colab, download the born.tsv file here (after class).  In both cases, adjust the path to `born.tsv` for your environment.

In [ ]:
!wget https://raw.githubusercontent.com/dbamman/anlp21/master/data/born.tsv

In [ ]:
trainingData=read_training("born.tsv")
for data in trainingData:
    print ('\t'.join([str(x) for x in data]))

Q1: Count the syntactic paths identified in the training data.  What are the two that are most frequently attested?

Q2: Write a function to read in a target file (containing one document per line) and a syntactic path and identify all people/places that are joined by that path. Hint: you can use the get_path functon defined above to retrieve the syntactic path between two tokens.

In [ ]:
def extract_relations(filename, target_path):
    
    """ Extract new relations from a file.
    Input: 
        - filename containing one document per line
        - target_path: the syntactic dependency path connecting the person entity to the place entity
    Output:
        - a list of (person, place, path, sentence) tuples in the same format returned from `read_training`.
    
    """
    data=[]
    
    # your code here
    
    return data

If you're executing this on Google Colab, download the wiki.bio.born.test.txt file here.

In [ ]:
!wget https://raw.githubusercontent.com/dbamman/anlp21/master/data/wiki.bio.born.test.txt

In [ ]:
new_examples=extract_relations("wiki.bio.born.test.txt", "nsubjpass->born<-prep<-pobj")
for data in new_examples:
    print ('\t'.join([str(x) for x in data]))

Q3: Execute `extract_relations` on `wiki.bio.born.test.txt` and the two most frequent paths identified in the training data above.